In [1]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-edlo09mc/farm-haystack_74017494752c4ffcbff45cf0de31209e
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-edlo09mc/farm-haystack_74017494752c4ffcbff45cf0de31209e
  Resolved https://github.com/deepset-ai/haystack.git to commit d2bba4935b2ccfa7ef875815a4a1bf98afcedbc1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done

In [2]:
!cp /content/drive/MyDrive/articles1.csv /content/

In [3]:
import pandas as pd

data_path = '/content/articles1.csv'
data = pd.read_csv(data_path)
data = list(data['content'].values)
data = data[:100]

In [4]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

In [5]:
docs = []

for value in data:

  doc = {
      "content":value
  }

  docs.append(doc)

document_store.write_documents(docs)

Writing Documents:   0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store = document_store,
    embedding_model = 'sentence-transformers/multi-qa-mpnet-base-dot-v1',
    model_format = 'sentence_transformers'
)

document_store.update_embeddings(retriever)

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Updating Embedding:   0%|          | 0/100 [00:00<?, ? docs/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline

reader = FARMReader(model_name_or_path = 'deepset/roberta-base-squad2', use_gpu = True)
pipeline = ExtractiveQAPipeline(reader, retriever)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [8]:
from haystack.utils import print_answers

prediction = pipeline.run(
    query = 'Who is Queen Elizabeth Husband?', params = {"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

answer = prediction['answers'][0]
print(answer.answer, round(answer.score,2))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples:   0%|          | 0/3 [00:00<?, ? Batches/s]

Prince Philip 0.99
